### Tensorboard and data setup

In [0]:
# Data, clone git
!git clone https://ashencz:@github.com/AshenCZ/deep-learning.git

# Change dir to the project
# TODO: Change directory of project
%cd ./deep-learning/mnist_comp
!ls

In [0]:
# Tensorboard

!git clone https://github.com/mixuala/colab_utils.git

import os
import colab_utils.tboard

# set paths
ROOT = %pwd
LOG_DIR = os.path.join(ROOT, 'logs')
print(ROOT, LOG_DIR)

# will install `ngrok`, if necessary
# will create `log_dir` if path does not exist
colab_utils.tboard.launch_tensorboard( bin_dir=ROOT, log_dir=LOG_DIR )

In [0]:
# Find out how much GPU memory you have left

# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

## Code

In [0]:
#!/usr/bin/env python3
import numpy as np
import tensorflow as tf

class Args(object):
  pass

### Network

In [0]:
# Network

class Network:
    WIDTH = 28
    HEIGHT = 28
    LABELS = 10

    def __init__(self, threads, seed=42):
        # Create an empty graph and a session
        graph = tf.Graph()
        graph.seed = seed
        self.session = tf.Session(graph = graph, config=tf.ConfigProto(inter_op_parallelism_threads=threads,
                                                                       intra_op_parallelism_threads=threads))

    def construct(self, args):
        with self.session.graph.as_default():
            # Inputs
            self.images = tf.placeholder(tf.float32, [None, self.WIDTH, self.HEIGHT, 1], name="images")
            self.labels = tf.placeholder(tf.int64, [None], name="labels")
            self.is_training = tf.placeholder(tf.bool, [], name="is_training")

            previous_layer = self.images

            layers = args.cnn.split(",")
            for l in layers:
                params = l.split("-")
                if l[0] == 'C':
                    filters = int(params[1])
                    kernel_size = int(params[2])
                    stride = int(params[3])
                    padding_param = params[4]
                    # print("Convo"+str(filters)+str(kernel_size)+str(stride)+str(padding_param))
                    next_layer = tf.layers.conv2d(previous_layer, filters, kernel_size, stride, padding=padding_param, activation=tf.nn.relu, name="Convo"+str(filters)+str(kernel_size)+str(stride)+str(padding_param))
                elif l[0] == 'M':
                    kernel_size = int(params[1])
                    stride = int(params[2])
                    # print("MaxPool"+str(kernel_size)+str(stride))
                    next_layer = tf.layers.max_pooling2d(previous_layer, kernel_size, stride, name="MaxPool"+str(kernel_size)+str(stride))
                elif l[0] == 'F':
                    next_layer = tf.layers.flatten(previous_layer, name="Flatten")
                    # print("Flatten")
                elif l[0] == 'R':
                    size = int(params[1])
                    next_layer = tf.layers.dense(previous_layer, size, activation=tf.nn.relu, name="dense"+str(size))
                    # print("dense"+str(size))
                elif l[0] == 'D':
                    # rate = float(params[1])
                    next_layer = tf.layers.dropout(inputs=previous_layer, rate=0.4, training=self.is_training)

                else:
                    raise Exception("What.")
                previous_layer = next_layer

            output_layer = tf.layers.dense(previous_layer, self.LABELS, activation=None, name="output_layer")
            self.predictions = tf.argmax(output_layer, axis=1)

            # Training
            loss = tf.losses.sparse_softmax_cross_entropy(self.labels, output_layer, scope="loss")
            global_step = tf.train.create_global_step()
            self.training = tf.train.AdamOptimizer().minimize(loss, global_step=global_step, name="training")

            # Summaries
            self.accuracy = tf.reduce_mean(tf.cast(tf.equal(self.labels, self.predictions), tf.float32))
            summary_writer = tf.contrib.summary.create_file_writer(args.logdir, flush_millis=10 * 1000)
            self.summaries = {}
            with summary_writer.as_default(), tf.contrib.summary.record_summaries_every_n_global_steps(100):
                self.summaries["train"] = [tf.contrib.summary.scalar("train/loss", loss),
                                           tf.contrib.summary.scalar("train/accuracy", self.accuracy)]
            with summary_writer.as_default(), tf.contrib.summary.always_record_summaries():
                for dataset in ["dev", "test"]:
                    self.summaries[dataset] = [tf.contrib.summary.scalar(dataset + "/loss", loss),
                                               tf.contrib.summary.scalar(dataset + "/accuracy", self.accuracy)]

            # Initialize variables
            self.session.run(tf.global_variables_initializer())
            with summary_writer.as_default():
                tf.contrib.summary.initialize(session=self.session, graph=self.session.graph)

    def train(self, images, labels):
        self.session.run([self.training, self.summaries["train"]], {self.images: images, self.labels: labels, self.is_training: True})

    def evaluate(self, dataset, images, labels):
        return self.session.run([self.predictions, self.accuracy, self.summaries[dataset]], {self.images: images, self.labels: labels, self.is_training: False})


### Training

In [0]:
if __name__ == "__main__":
    import argparse
    import datetime
    import os
    import re

    # Fix random seed
    np.random.seed(42)

    # TODO: Arguments
    args = Args()
    args.epochs = 1
    args.threads = 4
    
    args.batch_size = 50
    args.cnn = "C-32-5-1-same,M-2-2,C-64-5-1-same,M-2-2,F,R-1024,D,R-10"    

    # Create logdir name
    args.logdir = "logs/{}-{}-{}".format(
        "./",
        datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S"),
        ",".join(("{}={}".format(re.sub("(.)[^_]*_?", r"\1", key), value) for key, value in sorted(vars(args).items())))
    )
    if not os.path.exists("logs"): os.mkdir("logs") # TF 1.6 will do this by itself
    
    # Load the data
    from tensorflow.examples.tutorials import mnist
    mnist = mnist.input_data.read_data_sets("data", reshape=False, seed=42)

    # Construct the network
    network = Network(threads=args.threads)
    network.construct(args)

    # Train
    for i in range(args.epochs):
        while mnist.train.epochs_completed == i:
            images, labels = mnist.train.next_batch(args.batch_size)
            network.train(images, labels)

        _, acc, _  = network.evaluate("dev", mnist.validation.images, mnist.validation.labels)
        print(i," Dev: {:.2f}".format(100 * acc))


### Test set evaluation

In [0]:
# Generate the predictions
# TODO: Change predictions
predictions, _, _ = network.evaluate("test", mnist.test.images, mnist.test.labels)

# Open the file
time = datetime.datetime.now().time()
last_file_name = "output_test_"+str(time.hour)+str(time.minute)+".txt"
utf8stdout = open(last_file_name, 'w', encoding='utf-8', closefd=True)

# Print the results into the file
# TODO: Change format of output
for label in predictions:
  print(label, file=utf8stdout)

# Finished
print("File finished: ", last_file_name)
utf8stdout.close()

In [0]:
# Download the ouput data
from google.colab import files
files.download(last_file_name)

## The rest

In [0]:
# Blank for tests
!ls
# !kill -9 -1